In [1]:
!pip install pycuda

     |████████████████████████████████| 1.6MB 10.5MB/s 
     |████████████████████████████████| 71kB 9.8MB/s 
     |████████████████████████████████| 81kB 8.4MB/s 
  Created wheel for pycuda: filename=pycuda-2020.1-cp36-cp36m-linux_x86_64.whl size=620887 sha256=2f6098395305b87e2127ade39fe5c1245c5f278f1170f785818abb489a7e7ff8
  Stored in directory: /root/.cache/pip/wheels/8f/78/d1/5bb826f81d9d490297a348d818ff3ee6dd6f2075b06dde6ea0
  Created wheel for pytools: filename=pytools-2020.4.3-py2.py3-none-any.whl size=61374 sha256=b0f9699c6e75f486ea1d73db7f3895c69d79e47afa145259c809acc7e2c7c8a1
  Stored in directory: /root/.cache/pip/wheels/af/c7/81/a22edb90b0b09a880468b2253bb1df8e9f503337ee15432c64
Successfully built pycuda pytools


In [3]:
import pycuda.autoinit

from pycuda.tools import make_default_context
make_default_context().get_device().name()

'Tesla T4'

In [4]:
import numpy as np
from numpy import linalg as la
from pycuda import driver, compiler, gpuarray, tools
import pycuda.autoinit
import time

m_size = 100

def matmul_GPU(a_gpu,b_gpu,m_size=m_size):
    kernel_code_template = """
    __global__ void MatrixMulKernel(float *A, float *B, float *C)
    {

      const uint wA = %(m_size)s;
      const uint wB = %(m_size)s;

      const uint bx = blockIdx.x;
      const uint by = blockIdx.y;

      const uint tx = threadIdx.x;
      const uint ty = threadIdx.y;

      const uint aBegin = wA * %(b_size)s * by;
      const uint aEnd = aBegin + wA - 1;

      const uint aStep = %(b_size)s;

      const uint bBegin = %(b_size)s * bx;
      const uint bStep = %(b_size)s * wB;

      float Csub = 0;

      for (int a = aBegin, b = bBegin;
           a <= aEnd;
           a += aStep, b += bStep)
        {

          __shared__ float As[%(b_size)s][%(b_size)s];

          __shared__ float Bs[%(b_size)s][%(b_size)s];

          As[ty][tx] = A[a + wA * ty + tx];
          Bs[ty][tx] = B[b + wB * ty + tx];

          __syncthreads();
          for (int k = 0; k < %(b_size)s; ++k)
            Csub += As[ty][k] * Bs[k][tx];
          __syncthreads();
        }

      const uint c = wB * %(b_size)s * by + %(b_size)s * bx;
      C[c + wB * ty + tx] = Csub;
    }
    """

    t_size = 2
    b_size = t_size

    kernel_code = kernel_code_template % {
        'm_size': m_size,
        'b_size': b_size,
        }

    mod = compiler.SourceModule(kernel_code)
    
    c_gpu = gpuarray.empty((m_size, m_size), np.float32)

    matrixmul = mod.get_function("MatrixMulKernel")

    matrixmul(
        a_gpu, b_gpu,
        c_gpu,
        grid = (m_size // t_size, m_size // t_size),
        block = (t_size, t_size, 1),
        )

    return c_gpu


def matmul_CPU(matrix1, matrix2):
    rmatrix = np.zeros(shape=(matrix1.shape[0], matrix2.shape[1]))
    for i in range(len(matrix1)):
        for j in range(len(matrix2[0])):
            for k in range(len(matrix2)):
                rmatrix[i][j] += matrix1[i][k] * matrix2[k][j]
    return rmatrix

In [5]:
cpu_time = []
gpu_time = []
diffs = []

for size in [128, 256, 512]:
    a_cpu = np.random.randn(size, size).astype(np.float32)
    b_cpu = np.random.randn(size, size).astype(np.float32)

    print("Размерность матрицы:", size)
    
    startCPU = time.time()
    c_cpu = matmul_CPU(a_cpu, b_cpu)
    endCPU = time.time()
    timeCPU = endCPU -startCPU

    print("время на CPU:", timeCPU)
    cpu_time.append(timeCPU)

    a_gpu = gpuarray.to_gpu(a_cpu)
    b_gpu = gpuarray.to_gpu(b_cpu)

    startGPU = time.time()
    c_gpu = matmul_GPU(a_gpu, b_gpu, size)
    endGPU = time.time()
    timeGPU = endGPU-startGPU

    print("время на GPU:", timeGPU)
    gpu_time.append(timeGPU)

    differensetime = timeCPU-timeGPU
    print("CPU-GPU:", differensetime)
    diffs.append(differensetime)
    
    print ("___________________________")

Размерность матрицы: 128
время на CPU: 2.354503631591797
время на GPU: 0.9202377796173096
CPU-GPU: 1.4342658519744873
___________________________
Размерность матрицы: 256
время на CPU: 19.08896255493164
время на GPU: 0.27205991744995117
CPU-GPU: 18.81690263748169
___________________________
Размерность матрицы: 512
время на CPU: 153.40163278579712
время на GPU: 0.29764509201049805
CPU-GPU: 153.10398769378662
___________________________
